# Implement standard map-based idioms to filter and display

Steps:

  1. Display the basic map.
  
  2. Read the geocoded shape file.
  
  3. Create markers (to be clustered) for all the businesses in the two zip codes. Create selection for all business and put it on the map.
  
  4. Read the BID shape files.
  
  5. Select the El Cajon BID.
  
  6. Find the businesses within the BID poly.  Create a second clustering for these buisness and put them on the map.
  
Note as you go through these steps the map display is changing.


https://www.sandiego.gov/economic-development/about/bids

In [ ]:
import geopandas as gpd
from ipywidgets import HTML, Layout
from ipyleaflet import (Map, MarkerCluster, LayersControl, CircleMarker,
                       basemaps, basemap_to_tiles)

from tqdm import tqdm

In [ ]:
imagery = basemap_to_tiles(basemaps.Esri.WorldImagery)
imagery.base = True
osm = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
osm.base = True


map_display = Map(center=(32.715, -117.1625), zoom=12,
                  layers=[imagery, osm],
                  layout=Layout(height="700px"),
                  scroll_wheel_zoom=True)

map_display.add_control(LayersControl())
map_display

##  All businesses

In [ ]:
my_gdf = gpd.read_file("../data/ecb.shp").reset_index()

In [ ]:
len(my_gdf)

In [ ]:
my_gdf.info()

In [ ]:
valid_geo_gdf = my_gdf.dropna(subset=['geometry']).reset_index()

In [ ]:
len(my_gdf) - len(valid_geo_gdf)

## Build the markers and put them on the map

In [ ]:
business = list()
for i, r in tqdm(valid_geo_gdf.iterrows()):
    marker = CircleMarker(location=(r.geometry.y, r.geometry.x), radius=5, stroke=False, fill_color="blue", fill_opacity=1.0)
    msg = HTML()
    msg.value = f"{r['DBA NAME']}"
    marker.popup = msg
    business.append(marker)
    valid_geo_gdf.iloc[i]['marker'] = marker
valid_geo_gdf['marker'] = business

In [ ]:
business_cluster = MarkerCluster(markers=business, name='Businesses')
map_display.add_layer(business_cluster)

#map_display.add_control(LayersControl())

## Read the BID shape file

In [ ]:
bids_gdf = gpd.read_file("../data/bids/bids_datasd.shp")

bids_gdf = bids_gdf.to_crs(epsg=4326)

In [ ]:
bids_gdf

In [ ]:
ecb_bid = bids_gdf.query(f"name == 'El Cajon Boulevard Central'").reset_index()

In [ ]:
ecb_bid

## Filter based on the polygon

In [ ]:
ecb_biz_gdf = valid_geo_gdf[valid_geo_gdf.geometry.within(ecb_bid.iloc[0].geometry)]

In [ ]:
len(ecb_biz_gdf)

In [ ]:
ecb_biz_gdf.columns

In [ ]:
ecb_biz_gdf = ecb_biz_gdf.drop(columns=['level_0']).reset_index()

## Create the second set up businesses for the map

In [ ]:
ecb_business = list()
for i, r in tqdm(ecb_biz_gdf.iterrows()):
    marker = CircleMarker(location=(r.geometry.y, r.geometry.x), radius=5, stroke=False, fill_color="blue", fill_opacity=1.0)
    msg = HTML()
    msg.value = f"{r['DBA NAME']}"
    marker.popup = msg
    ecb_business.append(marker)
    ecb_biz_gdf.iloc[i]['marker'] = marker
ecb_biz_gdf['marker'] = ecb_business

In [ ]:
ecb_business_cluster = MarkerCluster(markers=ecb_business, name='Boulevard Businesses')
map_display.add_layer(ecb_business_cluster)